In [4]:
import pandas as pd                    # data package
import matplotlib.pyplot as plt        # graphics module  
import datetime as dt                  # date and time module
import numpy as np                     # foundation for pandas 

In [228]:
data_file = "./data/WIOT2014_Nov16_ROW.xlsb" 
# Just download as is from WIOT and the hope is to have the file setup so things are nice and clean

df = pd.read_excel(data_file, engine='pyxlsb')

traded = {"r1", "r2", "r3", "r4", "r5", "r6", "r7", "r8", "r9", "r10","r11","r12","r13","r14","r15","r16","r17","r18","r19","r20","r21","r22"}

total = {"r65"}

country_list = { 'AUS', 'AUT', 'BEL', 'BRA', 'CAN', 'CHE', 'CHN', 'DEU', 'DNK', 'ESP', 'FIN', 'FRA', 'GBR', 'GRC', 
               'IND', 'IRL', 'ITA', 'JPN', 'KOR', 'MEX', 'NLD', 'NOR', 'PRT', 'SWE', 'TWN', 'USA'}

all_country_list = {'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHN', 'CYP',
       'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC',
       'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'ITA', 'JPN', 'KOR', 'LTU',
       'LUX', 'LVA', 'MEX', 'MLT', 'NLD', 'NOR', 'POL', 'PRT', 'ROU',
       'RUS', 'SVK', 'SVN', 'SWE', 'TUR', 'TWN', 'USA', 'ROW'}

In [229]:
df.head()

,Intercountry Input-Output Table,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 2679,Unnamed: 2680,Unnamed: 2681,Unnamed: 2682,Unnamed: 2683,Unnamed: 2684,Unnamed: 2685,Unnamed: 2686,Unnamed: 2687,Unnamed: 2688
0,"43 countries, in current prices",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(industry-by-industry),NaN,NaN,NaN,A01,A02,A03,B,C10-C12,C13-C15,...,CONS_np,CONS_g,GFCF,INVEN,CONS_h,CONS_np,CONS_g,GFCF,INVEN,GO
2,(millions of US$),NaN,NaN,NaN,"Crop and animal production, hunting and relate...",Forestry and logging,Fishing and aquaculture,Mining and quarrying,"Manufacture of food products, beverages and to...","Manufacture of textiles, wearing apparel and l...",...,Final consumption expenditure by non-profit or...,Final consumption expenditure by government,Gross fixed capital formation,Changes in inventories and valuables,Final consumption expenditure by households,Final consumption expenditure by non-profit or...,Final consumption expenditure by government,Gross fixed capital formation,Changes in inventories and valuables,Total output
3,NaN,NaN,NaN,NaN,AUS,AUS,AUS,AUS,AUS,AUS,...,USA,USA,USA,USA,ROW,ROW,ROW,ROW,ROW,TOT
4,NaN,NaN,NaN,NaN,c1,c2,c3,c4,c5,c6,...,c58,c59,c60,c61,c57,c58,c59,c60,c61,c62


In [230]:
total_df = pd.DataFrame([])

for xxx in country_list:
    
    condition = ( df.iloc[4,:] == "c57") & (df.iloc[3,:] == xxx)
    # this first looks for country xxx and total final consumption expenditures
    
    row_condition = ( df.iloc[:,3].isin(traded) ) & (df.iloc[:,2].isin(all_country_list))
    # sum accross all traded goods, all countries, so how much traded consumption
    # not doing services here
    
    total_value = df.loc[row_condition,condition].sum()[0]
    # total value, all countries, traded consumption 
    
    for zzz in country_list:
        # now where did this come from, work through each exporter
    
        row_condition = ( df.iloc[:,3].isin(traded) ) & (df.iloc[:,2] == zzz)
        # tradeed consumption comming from zzz
        
        foobar = {"Importer": [xxx], #importer
            "Exporter": [zzz], #exporter
            "Value": [ df.loc[row_condition, condition].sum()[0] ], # value
           "Share": [ df.loc[row_condition, condition].sum()[0] / total_value ] } #share of total
    
        total_df = pd.concat([total_df, pd.DataFrame(foobar)], axis = 0)

    not_rest_of_world = total_df[total_df["Importer"] == xxx].Value.sum()  # now figure out how much from not rest of world
    
    foo = {"Importer": [xxx],
           "Exporter": "ROW",
          "Value": [ total_value - not_rest_of_world  ], # the ROW value is total - not rest of world
          "Share": [ (total_value - not_rest_of_world) / total_value ] } # share
    
    total_df = pd.concat([total_df, pd.DataFrame(foo)], axis = 0) # appened it

In [221]:
total_df[total_df["Importer"] == "USA"]

,Importer,Exporter,Value,Share
0,USA,IRL,994.132398,0.000870
0,USA,CAN,34579.945182,0.030245
0,USA,DNK,579.777699,0.000507
0,USA,IND,1242.673747,0.001087
0,USA,NLD,1170.389335,0.001024
0,USA,TWN,4114.417312,0.003599
0,USA,PRT,131.359565,0.000115
0,USA,CHE,1913.616789,0.001674
0,USA,MEX,63352.968094,0.055412
0,USA,DEU,34916.188852,0.030540


In [227]:
total_df[total_df["Importer"] == "CHN"]

,Importer,Exporter,Value,Share
0,CHN,IRL,25.008335,0.000079
0,CHN,CAN,153.525037,0.000482
0,CHN,DNK,55.560471,0.000174
0,CHN,IND,86.085786,0.000270
0,CHN,NLD,133.978578,0.000421
0,CHN,TWN,556.558733,0.001748
0,CHN,PRT,1.530446,0.000005
0,CHN,CHE,58.542172,0.000184
0,CHN,MEX,20.708817,0.000065
0,CHN,DEU,516.687464,0.001623


In [167]:
df.loc[row_condition, condition].sum()

Unnamed: 2523    21921.624353
dtype: object